In [43]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Import Statsmodels
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller
from statsmodels.tools.eval_measures import rmse, aic

In [44]:
df = pd.read_csv('Final_data.csv')

In [45]:
df=df.dropna()

In [46]:
df

,Date,CPI,Median Annual Income,Population Estimate,Petrol Price,Diesel Price,Google Search Index,All Cars,ULEV,Grants,Charging Points
0,2011-10-31,94.5,26719.28962,430572.0765,1.3473,1.4027,23.0,206427.0,31.0,0.000000e+00,0.0
1,2011-11-30,94.6,26652.97814,430979.3716,1.3317,1.4110,22.0,207544.0,31.0,0.000000e+00,0.0
2,2011-12-31,95.1,26584.45628,431400.2432,1.3241,1.4077,12.0,208700.0,31.0,0.000000e+00,0.0
3,2012-01-31,94.6,26515.93443,431821.1148,1.3410,1.4245,21.0,209690.0,31.0,0.000000e+00,0.0
4,2012-02-29,95.1,26451.83333,432214.8333,1.3621,1.4392,26.0,210618.0,33.0,0.000000e+00,0.0
...,...,...,...,...,...,...,...,...,...,...,...
106,2020-08-31,108.6,32819.57260,465877.4575,1.1329,1.1818,65.0,182533.0,1719.0,1.722130e+06,206.0
107,2020-09-30,109.1,32779.95616,465880.2521,1.1330,1.1814,79.0,182451.0,1795.0,1.747088e+06,209.0
108,2020-10-31,109.1,32739.01918,465883.1397,1.1314,1.1808,72.0,182366.0,1874.0,1.772879e+06,212.0
109,2020-11-30,108.9,32699.40274,465885.9342,1.1261,1.1741,85.0,182284.0,1919.0,1.797837e+06,215.0


In [47]:
df['Date']=pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)

In [48]:
from statsmodels.tsa.stattools import grangercausalitytests
maxlag=12
test = 'ssr_chi2test'
def grangers_causation_matrix(data, variables, test='ssr_chi2test', verbose=False):    
    """Check Granger Causality of all possible combinations of the Time series.
    The rows are the response variable, columns are predictors. The values in the table 
    are the P-Values. P-Values lesser than the significance level (0.05), implies 
    the Null Hypothesis that the coefficients of the corresponding past values is 
    zero, that is, the X does not cause Y can be rejected.

    data      : pandas dataframe containing the time series variables
    variables : list containing names of the time series variables.
    """
    df = pd.DataFrame(np.zeros((len(variables), len(variables))), columns=variables, index=variables)
    for c in df.columns:
        for r in df.index:
            test_result = grangercausalitytests(data[[r, c]], maxlag=maxlag, verbose=False)
            p_values = [round(test_result[i+1][0][test][1],4) for i in range(maxlag)]
            if verbose: print(f'Y = {r}, X = {c}, P Values = {p_values}')
            min_p_value = np.min(p_values)
            df.loc[r, c] = min_p_value
    df.columns = [var + '_x' for var in variables]
    df.index = [var + '_y' for var in variables]
    return df

grangers_causation_matrix(df, variables = df.columns)   

,CPI_x,Median Annual Income_x,Population Estimate_x,Petrol Price_x,Diesel Price_x,Google Search Index_x,All Cars_x,ULEV_x,Grants_x,Charging Points_x
CPI_y,1.0000,0.1585,0.0305,0.0000,0.0000,0.0637,0.0245,0.1845,0.0165,0.0033
Median Annual Income_y,0.0001,1.0000,0.0266,0.0013,0.0041,0.0000,0.0000,0.0121,0.0047,0.0003
Population Estimate_y,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
Petrol Price_y,0.0928,0.0639,0.0971,1.0000,0.0525,0.2123,0.3445,0.2109,0.0918,0.3612
Diesel Price_y,0.4431,0.2540,0.1810,0.3312,1.0000,0.1988,0.1441,0.2902,0.0236,0.3320
Google Search Index_y,0.0000,0.0001,0.0012,0.0038,0.0059,1.0000,0.0293,0.0001,0.0000,0.0000
All Cars_y,0.0009,0.0000,0.0000,0.0001,0.0000,0.0026,1.0000,0.0000,0.0001,0.0001
ULEV_y,0.1302,0.0001,0.1302,0.1496,0.0154,0.0334,0.0000,1.0000,0.1058,0.0000
Grants_y,0.0226,0.2978,0.0000,0.0000,0.0000,0.0000,0.0000,0.0983,1.0000,0.0000
Charging Points_y,0.0272,0.2389,0.0000,0.0000,0.0000,0.0000,0.0000,0.0908,0.0000,1.0000
